# feedback-prize-effectiveness

## import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

import pandas as pd

import torch
import transformers
import datasets

import importlib
#importlib.reload(load_data)


: 

In [ ]:
python -m pip install ipykernel -U --force-reinstall

: 

In [ ]:
is_kaggle=os.getenv('KAGGLE_KERNEL_RUN_TYPE', '')

if not is_kaggle:

    sys.path.append('../')

    from src.utils import config
    #from src.utils import logger
    from src.data import load_data

    from src.models.huggingface_transformers import BERT

    from src.training.huggingface_transformers import trainer

: 

In [ ]:
#logger=logger.Logger('test_log')

: 

In [ ]:
CFG=config.CFG(competition_name='feedback-prize-effectiveness')
CFG.debug=True
CFG.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

: 

In [ ]:
CFG.data_path

: 

## data

In [ ]:
if not CFG.is_kaggle:
    load_data.download_data(CFG.data_path+CFG.competition_name+'/',CFG.competition_name)

: 

In [ ]:
train_csv,test_csv,sample_submission_csv=load_data.load_data(CFG.data_path+CFG.competition_name+'/',debug=CFG.debug)

: 

In [ ]:
train_csv

: 

In [ ]:
test_csv

: 

## feature

In [ ]:
model_name='RoBERTa'

training=True

if training:
    pretrained_model='roberta-base'
    #pretrained_model=CFG.output_path+''
else :
    pretrained_model=CFG.weights_path+model_name+'/'
    

model=BERT.transformers_RoBERTa(pretrained_path=pretrained_model,device=CFG.device)

: 

In [ ]:
train_csv['input'] = train_csv.discourse_type + " " + model.tokenizer.sep_token + " " + train_csv.discourse_text
test_csv['input'] = test_csv.discourse_type + " " + model.tokenizer.sep_token + " " + test_csv.discourse_text

: 

In [ ]:
train_csv['label']=train_csv['discourse_effectiveness']
train_csv = train_csv.replace({"label": {"Ineffective": 0, "Adequate": 1, "Effective": 2}})
#train_csv = train_csv.rename(columns = {"discourse_effectiveness": "label"})

: 

### train val split

In [ ]:
essay_ids={'train':None,'val':None}
essay_ids['train'],essay_ids['val']=load_data.split(train_csv.essay_id.unique())

: 

In [ ]:
train_csv['train_val']=train_csv.essay_id.isin(essay_ids['train']).replace({True: 'train',False: 'val'})

: 

In [ ]:
df={'train':None,'val':None,'test':None}
df['train']=train_csv[train_csv['train_val']=='train']
df['val']=train_csv[train_csv['train_val']=='val']
df['test']=test_csv

: 

## training

In [ ]:
dataset={'train':None,'val':None,'test':None}

dataset['train'] = datasets.Dataset.from_pandas(df['train'])
dataset['val'] = datasets.Dataset.from_pandas(df['val'])
dataset['test'] = datasets.Dataset.from_pandas(df['test'])

: 

In [ ]:
df['val'].keys()

: 

In [ ]:
dataset

: 

In [ ]:
dataset['train'][0]

: 

In [ ]:
model.tokenizer(dataset['train'][0]["input"], max_length=512, truncation=True, padding="max_length")

: 

In [ ]:
def f(x):return model.tokenizer(x['input'],truncation=True)

dataset['train']=dataset['train'].map(
    f,
    batched=True,
    remove_columns=(
        'discourse_id',
        'essay_id',
        'discourse_text',
        'discourse_type',
        'discourse_effectiveness',
        'input',
        '__index_level_0__',#
    )
)
dataset['val']=dataset['val'].map(
    f,
    batched=True,
    remove_columns=(
        'discourse_id',
        'essay_id',
        'discourse_text',
        'discourse_type',
        'discourse_effectiveness',
        'input',
        '__index_level_0__',#
    )
)
dataset['test']=dataset['test'].map(
    f,
    batched=True,
    remove_columns=(
        'discourse_id',
        'essay_id',
        'discourse_text',
        'discourse_type',
        #'discourse_effectiveness',
        'input',
        #'__index_level_0__',#
    )
)

: 

In [ ]:
dataset

: 

In [ ]:
datasetdict=datasets.DatasetDict({
    "train":dataset['train'],
    "val": dataset['val'],
})

: 

In [ ]:
trainer = trainer.transformers_get_trainer(CFG.output_path+model_name+'/',model.model,model.tokenizer,datasetdict)

: 

In [ ]:
if training:
    trainer.train()
    #trainer.save_model(output_dir=CFG.output_path+model_name+'/')

: 

In [ ]:
trainer.state.log_history

: 

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(
    [x['epoch'] for x in trainer.state.log_history if 'loss' in x.keys()],
    [x['loss'] for x in trainer.state.log_history if 'loss' in x.keys()],
    label='train',
)

plt.plot(
    [x['epoch'] for x in trainer.state.log_history if 'eval_loss' in x.keys()],
    [x['eval_loss'] for x in trainer.state.log_history if 'eval_loss' in x.keys()],
    label='val'
)

plt.legend()
plt.plot()

: 

## prediction

In [ ]:
preds=torch.Tensor(trainer.predict(dataset['test']).predictions)
preds = torch.nn.functional.softmax(preds,dim=1).numpy().astype(float)
preds

: 

In [ ]:
submission=sample_submission_csv.copy()
submission['Ineffective'] = preds[:,0]
submission['Adequate'] = preds[:,1]
submission['Effective'] = preds[:,2]
submission

: 

In [ ]:
submission.to_csv('submission.csv',index=False)

: 

In [ ]:
#sample_submission_csv.to_csv('submission.csv',index=False)

: 